In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Layer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras import regularizers



grade_to_ordinal_dummy_11_categories = {
    'V4':[0,0,0,0,0,0,0,0,0,0],
    'V5':[1,0,0,0,0,0,0,0,0,0],
    'V6':[1,1,0,0,0,0,0,0,0,0],
    'V7':[1,1,1,0,0,0,0,0,0,0],
    'V8':[1,1,1,1,0,0,0,0,0,0],
    'V9':[1,1,1,1,1,0,0,0,0,0],
    'V10':[1,1,1,1,1,1,0,0,0,0],
    'V11':[1,1,1,1,1,1,1,0,0,0],
    'V12':[1,1,1,1,1,1,1,1,0,0],
    'V13':[1,1,1,1,1,1,1,1,1,0],
    'V14':[1,1,1,1,1,1,1,1,1,1]
}



class Conv2D_centered(Layer):
    # Custom convolution that acts non-trivially only when the center is nonzero (there is a hold there).
    # As explained in the Powerpoint, the number of holds is upsampled to be equal (twelve holds total) in all problems.
    # This way, the network can be expected to perform an averaging over the difficulty of the hold at the center, and
    # the local environment around the holds in each problem. If the environment is, on average, crowded, the problem 
    # should be easier, while if it's not crowded it should be harder. A simple filter with a 1 at the center and negative
    # isotropic values that fall off in magnitude with the distance would already accomplish a very rough measure of difficulty,
    # assuming the rest of the network accomplishes a sort of average of these filtered values over all holds.
    
    def __init__(self, kernel_shape):
        super(Conv2D_centered, self).__init__()
        self.kernel_shape = kernel_shape
        
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=self.kernel_shape,
                                      initializer='uniform',
                                      trainable=True)
        self.shape = input_shape
        super(Conv2D_centered, self).build(input_shape)  # Be sure to call this at the end

    def call(self, matrix):
        filtered = []
        for m in range(self.kernel_shape[0]):
            filtered.append(tf.reduce_sum(tf.reduce_sum(matrix*self.kernel[m], axis=-1), axis=-1))
        filtered_concatenated =  tf.concat(filtered, axis=1)
        return filtered_concatenated

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1]*self.kernel_shape[0])
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'kernel_shape': self.kernel_shape,
        })
        return config

    
def define_model():
    model = Sequential()
    model.add(Conv2D_centered((50,13,13)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l1(0.02)))
    model.add(Dense(10, activation='sigmoid'))
    # compile model
    model.compile(optimizer='adam', loss= 'bce', metrics=['mse'])
    return model

X_train = np.load('problems_train.npy')
X_test = np.load('problems_test.npy')
Y_train = np.load('grades_train.npy')
Y_test = np.load('grades_test.npy')


model = define_model()
history = model.fit(X_train, Y_train, epochs=150, batch_size=64, verbose=2)

binar = np.vectorize(lambda x: int(round(x)))
Y_test_predicted = binar(model.predict(X_test))

Epoch 1/150
277/277 - 1s - loss: 6.9626 - mse: 0.1274
Epoch 2/150
277/277 - 1s - loss: 0.4434 - mse: 0.0942
Epoch 3/150
277/277 - 1s - loss: 0.4373 - mse: 0.0941
Epoch 4/150
277/277 - 1s - loss: 0.4366 - mse: 0.0941
Epoch 5/150
277/277 - 1s - loss: 0.4356 - mse: 0.0941
Epoch 6/150
277/277 - 1s - loss: 0.4359 - mse: 0.0941
Epoch 7/150
277/277 - 1s - loss: 0.4365 - mse: 0.0941
Epoch 8/150
277/277 - 1s - loss: 0.4359 - mse: 0.0941
Epoch 9/150
277/277 - 1s - loss: 0.4360 - mse: 0.0941
Epoch 10/150
277/277 - 1s - loss: 0.4360 - mse: 0.0941
Epoch 11/150
277/277 - 1s - loss: 0.4371 - mse: 0.0941
Epoch 12/150
277/277 - 1s - loss: 0.4362 - mse: 0.0941
Epoch 13/150
277/277 - 1s - loss: 0.4366 - mse: 0.0941
Epoch 14/150
277/277 - 1s - loss: 0.4367 - mse: 0.0942
Epoch 15/150
277/277 - 1s - loss: 0.4365 - mse: 0.0941
Epoch 16/150
277/277 - 1s - loss: 0.4368 - mse: 0.0942
Epoch 17/150
277/277 - 1s - loss: 0.4377 - mse: 0.0942
Epoch 18/150
277/277 - 1s - loss: 0.4368 - mse: 0.0941
Epoch 19/150
277/27

277/277 - 2s - loss: 0.3594 - mse: 0.0621
Epoch 150/150
277/277 - 2s - loss: 0.3660 - mse: 0.0626


In [2]:
# Evaluate accuracy

confusion = np.zeros((11,11))
labels = []
for key,val in grade_to_ordinal_dummy_11_categories.items():
    labels.append(val)

for i in range(11):
    for j in range(11):
        for k in range(len(Y_test)):
            if np.array_equal(Y_test[k],labels[i]) and np.array_equal(Y_test_predicted[k],labels[j]):
                confusion[j,i] += 1
f_1 = np.zeros(11)

for i in range(11):
    precision = confusion[i,i]/np.sum(confusion[i,:])
    recall = confusion[i,i]/np.sum(confusion[:,i])
    if recall + precision > 0:
        f_1[i] = 2*precision*recall/(precision+recall)
    else:
        f_1[i] = 0
        
print("F_1 scores over the 11 categories V4 - V14 are:")
print(f_1)
    
    
accuracy = 0
for i in range(len(Y_test)):
    if list(Y_test[i] - Y_test_predicted[i]) == [0,0,0,0,0,0,0,0,0,0]:
        accuracy += 1/len(Y_test)
print("Total accuracy is:")
print(accuracy)

print("AUC is:")
print(float(tf.keras.metrics.AUC(multi_label = True)(Y_test, Y_test_predicted)))

# Note:  if the machine learning is done with 6 categories (V4, V5, V6, V7, V8, V9+), the accuracy is about 42% and AUC 70%,
# as presented in the Powerpoint.  However, I wanted to report the machine learning of 11 categories (V4 - V14), since that way 
# it can be compared to http://cs229.stanford.edu/proj2017/final-reports/5232206.pdf (where CNN accuracy was 34%).  
# The accuracy I achieve is higher!

F_1 scores over the 11 categories V4 - V14 are:
[0.62002472 0.43494817 0.26822818 0.20308043 0.19180201 0.09312639
 0.         0.         0.         0.         0.        ]
Total accuracy is:
0.39101717305150324
AUC is:
0.5824015140533447


<ipython-input-2-358f53c553eb>:16: RuntimeWarning: invalid value encountered in double_scalars
  precision = confusion[i,i]/np.sum(confusion[i,:])
